### Advanced Housing Prices- Feature Engineering
The main aim of this project is to predict the house price based on various features

#### Dataset to downloaded from the below link
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data


#### Steps to follow for Feature Engineering Phashe

1. Missing values
2. Temporal variables (date time)
3. Categorical variables: remove rare labels
4. Scaling the values of the features to the same range

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [236]:
# Loading the dataset
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


(1459, 80)

In [6]:
'''
droping because this target columns in not in test data set,
I will add it will train data set when i separate the the combined data 
'''
target_df_column = df_train['SalePrice']
df_train.drop(['SalePrice'], axis = 1, inplace = True) 

In [7]:
target_df_column

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [8]:
#combine the two dataset

# Add a 'source' column to distinguish between train and test data
df_train['source'] = 'train'
df_test['source'] = 'test'

# Combine datasets
dataset = pd.concat([df_train, df_test], ignore_index=True)


In [9]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,source
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,train
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,train
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,train
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,train
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,train


In [10]:
dataset.shape

(2919, 81)

## Missing values handle

### categorical missing value handing
1. we can drop rows and columns with missing values
2. we can replace missing value with specific value like 'missing'
3. we can replace missing value with the mode of each feature

In [13]:
cat_features = [feature for feature in dataset.columns if dataset[feature].dtypes == 'O' and feature != 'source']
cat_features

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [14]:
## finding the columns and percentages of missing values in categorical columns
cat_feature_with_null = [feature for feature in cat_features if dataset[feature].isnull().sum()>0]

for feature in cat_feature_with_null:
    print(f'{feature} = {dataset[feature].isnull().mean()*100} %')

MSZoning = 0.1370332305584104 %
Alley = 93.21685508735868 %
Utilities = 0.0685166152792052 %
Exterior1st = 0.0342583076396026 %
Exterior2nd = 0.0342583076396026 %
MasVnrType = 60.500171291538194 %
BsmtQual = 2.774922918807811 %
BsmtCond = 2.8091812264474134 %
BsmtExposure = 2.8091812264474134 %
BsmtFinType1 = 2.7064063035286057 %
BsmtFinType2 = 2.7406646111682083 %
Electrical = 0.0342583076396026 %
KitchenQual = 0.0342583076396026 %
Functional = 0.0685166152792052 %
FireplaceQu = 48.646796848235695 %
GarageType = 5.378554299417608 %
GarageFinish = 5.4470709146968135 %
GarageQual = 5.4470709146968135 %
GarageCond = 5.4470709146968135 %
PoolQC = 99.65741692360398 %
Fence = 80.4385063377869 %
MiscFeature = 96.40287769784173 %
SaleType = 0.0342583076396026 %


In [15]:
# Finding the columns with greater then 50% of null values
cat_col_To_drop = [feature for feature in cat_feature_with_null if dataset[feature].isnull().mean()*100 >50]
cat_col_To_drop

['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']

In [16]:
# droping columns
dataset.drop(cat_col_To_drop, axis = 1, inplace = True)
dataset.shape

(2919, 76)

In [17]:
## Replaing missing value mode
# Convert to sets and subtract
cat_feature_with_null = list(set(cat_feature_with_null) - set(cat_col_To_drop))
for feature in cat_feature_with_null:
    dataset[feature].replace(np.nan, dataset[feature].mode()[0], inplace = True)

#checking the null values
dataset[cat_feature_with_null].isnull().sum()


BsmtFinType2    0
GarageQual      0
BsmtFinType1    0
Exterior1st     0
Exterior2nd     0
BsmtQual        0
SaleType        0
BsmtCond        0
Functional      0
FireplaceQu     0
MSZoning        0
KitchenQual     0
GarageFinish    0
Electrical      0
Utilities       0
BsmtExposure    0
GarageCond      0
GarageType      0
dtype: int64

## Numerical missing value handling
* we already saw that all numerical values in skewed distribution,
* so, i will replace all missing value with the median of the feature

In [19]:
num_columns = [feature for feature in dataset.columns if dataset[feature].dtypes != 'O' and feature != 'source']
num_columns

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [20]:
## checking the missing value percentages
num_cols_nan = [feature for feature in num_columns if dataset[feature].isnull().sum()>0]

for feature in num_cols_nan:
    print(f'{feature} = {dataset[feature].isnull().mean()*100} %')

LotFrontage = 16.649537512846866 %
MasVnrArea = 0.7879410757108599 %
BsmtFinSF1 = 0.0342583076396026 %
BsmtFinSF2 = 0.0342583076396026 %
BsmtUnfSF = 0.0342583076396026 %
TotalBsmtSF = 0.0342583076396026 %
BsmtFullBath = 0.0685166152792052 %
BsmtHalfBath = 0.0685166152792052 %
GarageYrBlt = 5.4470709146968135 %
GarageCars = 0.0342583076396026 %
GarageArea = 0.0342583076396026 %


In [21]:
## replacing missing vlaues using median values of each features
for feature in num_cols_nan:
    dataset[feature].replace(np.nan, dataset[feature].median(), inplace =True)


In [22]:
## checking full data set missing if any have
dataset.isnull().sum().sum()

0

## Transforming Temporal variables
* we already saw at EDA phase if the year defference with the YrSold of other
   year data is high then price of the house will be low
* And for any unfortunate reason saleing price was low YrSold is up-to-date day by day
* Now i will transform the temporal feature to difference with YrSold

In [24]:
temporal = [feature for feature in num_columns if "Year" in feature or 'Yr' in feature]
temporal

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

In [25]:
#Transforming
for feature in temporal:
    if feature == 'YrSold':
        pass
    else:
        dataset[feature] = dataset['YrSold'] - dataset[feature]


In [26]:
dataset[temporal].head()

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,5,5,5.0,2008
1,31,31,31.0,2007
2,7,6,7.0,2008
3,91,36,8.0,2006
4,8,8,8.0,2008


## Handling rare categorical value
1. if a data set have categorical value of each categorical feature less than 1% then we should replace them with one
       common value like "Rare_var"
2. This work depends on each domain of project

In [28]:
'''
for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.01].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')

    '''

"\nfor feature in categorical_features:\n    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)\n    temp_df=temp[temp>0.01].index\n    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')\n\n    "

 ## Categorical to Numerical 

In [30]:
# importing labelEncoder 
from sklearn.preprocessing import LabelEncoder

In [31]:
categorical_features = [feature for feature in dataset.columns if dataset[feature].dtypes =='O' and feature != 'source']

In [32]:
# transforming form categroical to numerical features
le = LabelEncoder()
for feature in categorical_features:
    dataset[feature] = le.fit_transform(dataset[feature])

In [33]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,source
0,1,60,3,65.0,8450,1,3,3,0,4,0,5,2,2,0,5,7,5,5,5,1,1,12,13,196.0,2,4,2,2,3,3,2,706.0,5,0.0,150.0,856.0,1,0,1,4,856,854,0,1710,1.0,0.0,2,1,3,1,2,8,6,0,2,1,5.0,1,2.0,548.0,4,4,2,0,61,0,0,0,0,0,2,2008,8,4,train
1,2,20,3,80.0,9600,1,3,3,0,2,0,24,1,2,0,2,6,8,31,31,1,1,8,8,0.0,3,4,1,2,3,1,0,978.0,5,0.0,284.0,1262.0,1,0,1,4,1262,0,0,1262,0.0,1.0,2,0,3,1,3,6,6,1,4,1,31.0,1,2.0,460.0,4,4,2,298,0,0,0,0,0,0,5,2007,8,4,train
2,3,60,3,68.0,11250,1,0,3,0,4,0,5,2,2,0,5,7,5,7,6,1,1,12,13,162.0,2,4,2,2,3,2,2,486.0,5,0.0,434.0,920.0,1,0,1,4,920,866,0,1786,1.0,0.0,2,1,3,1,2,6,6,1,4,1,7.0,1,2.0,608.0,4,4,2,0,42,0,0,0,0,0,9,2008,8,4,train
3,4,70,3,60.0,9550,1,0,3,0,0,0,6,2,2,0,5,7,5,91,36,1,1,13,15,0.0,3,4,0,3,1,3,0,216.0,5,0.0,540.0,756.0,1,2,1,4,961,756,0,1717,1.0,0.0,1,0,3,1,2,7,6,1,2,5,8.0,2,3.0,642.0,4,4,2,0,35,272,0,0,0,0,2,2006,8,0,train
4,5,60,3,84.0,14260,1,0,3,0,2,0,15,2,2,0,5,8,5,8,8,1,1,12,13,350.0,2,4,2,2,3,0,2,655.0,5,0.0,490.0,1145.0,1,0,1,4,1145,1053,0,2198,1.0,0.0,2,1,4,1,2,9,6,1,4,1,8.0,1,3.0,836.0,4,4,2,192,84,0,0,0,0,0,12,2008,8,4,train


## Feature scalling
* we have to scale the data because all data distrubution is not in same range 

In [35]:
#getting columns for feature scale
feature_scale = [feature for feature in dataset.columns if feature not in ['Id', "source"]]
feature_scale

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch'

In [36]:
## importing MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(dataset[feature_scale])

MinMaxScaler()

In [37]:
# transforming 
scaled_data = scaler.transform(dataset[feature_scale])

In [38]:
# Convert the scaled data back to a DataFrame
data = pd.DataFrame(scaled_data, columns=feature_scale)

In [39]:
#checking the processed dataset
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.235294,0.75,0.150685,0.033420,1.0,1.0,1.0,0.0,1.0,0.0,0.208333,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.043796,0.112903,0.2,0.142857,0.857143,0.866667,0.12250,0.666667,1.0,0.4,0.666667,1.000000,1.000000,0.4,0.125089,1.0,0.0,0.064212,0.140098,0.2,0.0,1.0,1.0,0.109641,0.413559,0.0,0.259231,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.461538,1.0,0.00,0.5,0.2,0.652866,0.5,0.4,0.368280,1.0,1.0,1.0,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,1.0,0.8
1,0.000000,0.75,0.202055,0.038795,1.0,1.0,1.0,0.0,0.5,0.0,1.000000,0.125,0.285714,0.0,0.285714,0.555556,0.875,0.233577,0.532258,0.2,0.142857,0.571429,0.533333,0.00000,1.000000,1.0,0.2,0.666667,1.000000,0.333333,0.0,0.173281,1.0,0.0,0.121575,0.206547,0.2,0.0,1.0,1.0,0.194917,0.000000,0.0,0.174830,0.000000,0.5,0.50,0.0,0.375,0.333333,1.000000,0.307692,1.0,0.25,1.0,0.2,0.735669,0.5,0.4,0.309140,1.0,1.0,1.0,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,1.0,0.8
2,0.235294,0.75,0.160959,0.046507,1.0,0.0,1.0,0.0,1.0,0.0,0.208333,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.058394,0.129032,0.2,0.142857,0.857143,0.866667,0.10125,0.666667,1.0,0.4,0.666667,1.000000,0.666667,0.4,0.086109,1.0,0.0,0.185788,0.150573,0.2,0.0,1.0,1.0,0.123083,0.419370,0.0,0.273549,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.307692,1.0,0.25,1.0,0.2,0.659236,0.5,0.4,0.408602,1.0,1.0,1.0,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,1.0,0.8
3,0.294118,0.75,0.133562,0.038561,1.0,0.0,1.0,0.0,0.0,0.0,0.250000,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.671533,0.612903,0.2,0.142857,0.928571,1.000000,0.00000,1.000000,1.0,0.0,1.000000,0.333333,1.000000,0.0,0.038271,1.0,0.0,0.231164,0.123732,0.2,0.5,1.0,1.0,0.131695,0.366102,0.0,0.260550,0.333333,0.0,0.25,0.0,0.375,0.333333,0.666667,0.384615,1.0,0.25,0.5,1.0,0.662420,1.0,0.6,0.431452,1.0,1.0,1.0,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00,1.0,0.0
4,0.235294,0.75,0.215753,0.060576,1.0,0.0,1.0,0.0,0.5,0.0,0.625000,0.250,0.285714,0.0,0.714286,0.777778,0.500,0.065693,0.161290,0.2,0.142857,0.857143,0.866667,0.21875,0.666667,1.0,0.4,0.666667,1.000000,0.000000,0.4,0.116052,1.0,0.0,0.209760,0.187398,0.2,0.0,1.0,1.0,0.170342,0.509927,0.0,0.351168,0.333333,0.0,0.50,0.5,0.500,0.333333,0.666667,0.538462,1.0,0.25,1.0,0.2,0.662420,0.5,0.6,0.561828,1.0,1.0,1.0,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,1.0,0.8


In [40]:
# adding Id and source columns with dataset
dataset = pd.concat([dataset[["Id", "source"]].reset_index(drop=1), data[feature_scale]], axis =1)

In [41]:
dataset.head()


,Id,source,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,train,0.235294,0.75,0.150685,0.033420,1.0,1.0,1.0,0.0,1.0,0.0,0.208333,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.043796,0.112903,0.2,0.142857,0.857143,0.866667,0.12250,0.666667,1.0,0.4,0.666667,1.000000,1.000000,0.4,0.125089,1.0,0.0,0.064212,0.140098,0.2,0.0,1.0,1.0,0.109641,0.413559,0.0,0.259231,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.461538,1.0,0.00,0.5,0.2,0.652866,0.5,0.4,0.368280,1.0,1.0,1.0,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.0,0.090909,0.50,1.0,0.8
1,2,train,0.000000,0.75,0.202055,0.038795,1.0,1.0,1.0,0.0,0.5,0.0,1.000000,0.125,0.285714,0.0,0.285714,0.555556,0.875,0.233577,0.532258,0.2,0.142857,0.571429,0.533333,0.00000,1.000000,1.0,0.2,0.666667,1.000000,0.333333,0.0,0.173281,1.0,0.0,0.121575,0.206547,0.2,0.0,1.0,1.0,0.194917,0.000000,0.0,0.174830,0.000000,0.5,0.50,0.0,0.375,0.333333,1.000000,0.307692,1.0,0.25,1.0,0.2,0.735669,0.5,0.4,0.309140,1.0,1.0,1.0,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.0,0.363636,0.25,1.0,0.8
2,3,train,0.235294,0.75,0.160959,0.046507,1.0,0.0,1.0,0.0,1.0,0.0,0.208333,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.058394,0.129032,0.2,0.142857,0.857143,0.866667,0.10125,0.666667,1.0,0.4,0.666667,1.000000,0.666667,0.4,0.086109,1.0,0.0,0.185788,0.150573,0.2,0.0,1.0,1.0,0.123083,0.419370,0.0,0.273549,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.307692,1.0,0.25,1.0,0.2,0.659236,0.5,0.4,0.408602,1.0,1.0,1.0,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.0,0.727273,0.50,1.0,0.8
3,4,train,0.294118,0.75,0.133562,0.038561,1.0,0.0,1.0,0.0,0.0,0.0,0.250000,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.671533,0.612903,0.2,0.142857,0.928571,1.000000,0.00000,1.000000,1.0,0.0,1.000000,0.333333,1.000000,0.0,0.038271,1.0,0.0,0.231164,0.123732,0.2,0.5,1.0,1.0,0.131695,0.366102,0.0,0.260550,0.333333,0.0,0.25,0.0,0.375,0.333333,0.666667,0.384615,1.0,0.25,0.5,1.0,0.662420,1.0,0.6,0.431452,1.0,1.0,1.0,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.0,0.090909,0.00,1.0,0.0
4,5,train,0.235294,0.75,0.215753,0.060576,1.0,0.0,1.0,0.0,0.5,0.0,0.625000,0.250,0.285714,0.0,0.714286,0.777778,0.500,0.065693,0.161290,0.2,0.142857,0.857143,0.866667,0.21875,0.666667,1.0,0.4,0.666667,1.000000,0.000000,0.4,0.116052,1.0,0.0,0.209760,0.187398,0.2,0.0,1.0,1.0,0.170342,0.509927,0.0,0.351168,0.333333,0.0,0.50,0.5,0.500,0.333333,0.666667,0.538462,1.0,0.25,1.0,0.2,0.662420,0.5,0.6,0.561828,1.0,1.0,1.0,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.0,1.000000,0.50,1.0,0.8


In [85]:
## separate the combined dataset to train and test dataset
# Separate train and test datasets
processed_train = dataset[dataset['source'] == 'train'].drop(columns=['source'])
processed_test = dataset[dataset['source'] == 'test'].drop(columns=['source'])


In [89]:
processed_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,0.235294,0.75,0.150685,0.033420,1.0,1.0,1.0,0.0,1.0,0.0,0.208333,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.043796,0.112903,0.2,0.142857,0.857143,0.866667,0.122500,0.666667,1.0,0.4,0.666667,1.000000,1.000000,0.4,0.125089,1.0,0.000000,0.064212,0.140098,0.2,0.0,1.0,1.0,0.109641,0.413559,0.0,0.259231,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.461538,1.000000,0.00,0.5,0.2,0.652866,0.5,0.4,0.368280,1.0,1.0,1.0,0.000000,0.082210,0.000000,0.0,0.0,0.0,0.000000,0.090909,0.50,1.0,0.8
1,2,0.000000,0.75,0.202055,0.038795,1.0,1.0,1.0,0.0,0.5,0.0,1.000000,0.125,0.285714,0.0,0.285714,0.555556,0.875,0.233577,0.532258,0.2,0.142857,0.571429,0.533333,0.000000,1.000000,1.0,0.2,0.666667,1.000000,0.333333,0.0,0.173281,1.0,0.000000,0.121575,0.206547,0.2,0.0,1.0,1.0,0.194917,0.000000,0.0,0.174830,0.000000,0.5,0.50,0.0,0.375,0.333333,1.000000,0.307692,1.000000,0.25,1.0,0.2,0.735669,0.5,0.4,0.309140,1.0,1.0,1.0,0.209270,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.363636,0.25,1.0,0.8
2,3,0.235294,0.75,0.160959,0.046507,1.0,0.0,1.0,0.0,1.0,0.0,0.208333,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.058394,0.129032,0.2,0.142857,0.857143,0.866667,0.101250,0.666667,1.0,0.4,0.666667,1.000000,0.666667,0.4,0.086109,1.0,0.000000,0.185788,0.150573,0.2,0.0,1.0,1.0,0.123083,0.419370,0.0,0.273549,0.333333,0.0,0.50,0.5,0.375,0.333333,0.666667,0.307692,1.000000,0.25,1.0,0.2,0.659236,0.5,0.4,0.408602,1.0,1.0,1.0,0.000000,0.056604,0.000000,0.0,0.0,0.0,0.000000,0.727273,0.50,1.0,0.8
3,4,0.294118,0.75,0.133562,0.038561,1.0,0.0,1.0,0.0,0.0,0.0,0.250000,0.250,0.285714,0.0,0.714286,0.666667,0.500,0.671533,0.612903,0.2,0.142857,0.928571,1.000000,0.000000,1.000000,1.0,0.0,1.000000,0.333333,1.000000,0.0,0.038271,1.0,0.000000,0.231164,0.123732,0.2,0.5,1.0,1.0,0.131695,0.366102,0.0,0.260550,0.333333,0.0,0.25,0.0,0.375,0.333333,0.666667,0.384615,1.000000,0.25,0.5,1.0,0.662420,1.0,0.6,0.431452,1.0,1.0,1.0,0.000000,0.047170,0.268775,0.0,0.0,0.0,0.000000,0.090909,0.00,1.0,0.0
4,5,0.235294,0.75,0.215753,0.060576,1.0,0.0,1.0,0.0,0.5,0.0,0.625000,0.250,0.285714,0.0,0.714286,0.777778,0.500,0.065693,0.161290,0.2,0.142857,0.857143,0.866667,0.218750,0.666667,1.0,0.4,0.666667,1.000000,0.000000,0.4,0.116052,1.0,0.000000,0.209760,0.187398,0.2,0.0,1.0,1.0,0.170342,0.509927,0.0,0.351168,0.333333,0.0,0.50,0.5,0.500,0.333333,0.666667,0.538462,1.000000,0.25,1.0,0.2,0.662420,0.5,0.6,0.561828,1.0,1.0,1.0,0.134831,0.113208,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.50,1.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,0.235294,0.75,0.140411,0.030929,1.0,1.0,1.0,0.0,1.0,0.0,0.333333,0.250,0.285714,0.0,0.714286,0.555556,0.500,0.065693,0.145161,0.2,0.142857,0.857143,0.866667,0.000000,1.000000,1.0,0.4,0.666667,1.000000,1.000000,1.0,0.000000,1.0,0.000000,0.407962,0.155974,0.2,0.0,1.0,1.0,0.130015,0.336077,0.0,0.247362,0.000000,0.0,0.50,0.5,0.375,0.333333,1.000000,0.384615,1.000000,0.25,1.0,0.2,0.662420,0.5,0.4,0.309140,1.0,1.0,1.0,0.000000,0.05390

In [97]:
processed_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1460,1461,0.000000,0.50,0.202055,0.048246,1.0,1.0,1.000000,0.0,1.0,0.0,0.500000,0.125,0.285714,0.0,0.285714,0.444444,0.625,0.364964,0.822581,0.2,0.142857,0.857143,0.866667,0.0000,1.000000,1.0,0.2,1.000000,1.0,1.0,0.8,0.082920,0.6,0.094364,0.115582,0.144354,0.2,1.0,1.0,1.0,0.118042,0.000000,0.0,0.105878,0.0,0.0,0.25,0.0,0.250,0.333333,1.000000,0.230769,1.0,0.00,0.5,0.2,0.792994,1.0,0.2,0.490591,1.0,1.0,1.0,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0,1.0,0.8
1461,1462,0.000000,0.75,0.205479,0.060609,1.0,0.0,1.000000,0.0,0.0,0.0,0.500000,0.250,0.285714,0.0,0.285714,0.555556,0.625,0.386861,0.870968,0.6,0.142857,0.928571,0.933333,0.0675,1.000000,1.0,0.2,1.000000,1.0,1.0,0.0,0.163536,1.0,0.000000,0.173801,0.217512,0.2,1.0,1.0,1.0,0.208990,0.000000,0.0,0.187453,0.0,0.0,0.25,0.5,0.375,0.333333,0.666667,0.307692,1.0,0.00,0.5,0.2,0.802548,1.0,0.2,0.209677,1.0,1.0,1.0,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0,1.0,0.8
1462,1463,0.235294,0.75,0.181507,0.058566,1.0,0.0,1.000000,0.0,1.0,0.0,0.333333,0.250,0.285714,0.0,0.714286,0.444444,0.500,0.102190,0.225806,0.2,0.142857,0.857143,0.866667,0.0000,1.000000,1.0,0.4,0.666667,1.0,1.0,0.4,0.140149,1.0,0.000000,0.058647,0.151882,0.2,0.5,1.0,1.0,0.124764,0.339467,0.0,0.243971,0.0,0.0,0.50,0.5,0.375,0.333333,1.000000,0.307692,1.0,0.25,1.0,0.2,0.678344,0.0,0.4,0.323925,1.0,1.0,1.0,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0,1.0,0.8
1463,1464,0.235294,0.75,0.195205,0.040562,1.0,0.0,1.000000,0.0,1.0,0.0,0.333333,0.250,0.285714,0.0,0.714286,0.555556,0.625,0.094891,0.225806,0.2,0.142857,0.857143,0.866667,0.0125,1.000000,1.0,0.4,1.000000,1.0,1.0,0.4,0.106662,1.0,0.000000,0.138699,0.151555,0.2,0.0,1.0,1.0,0.124344,0.328329,0.0,0.239261,0.0,0.0,0.50,0.5,0.375,0.333333,0.666667,0.384615,1.0,0.25,0.5,0.2,0.675159,0.0,0.4,0.315860,1.0,1.0,1.0,0.252809,0.048518,0.0,0.0,0.000000,0.0,0.000000,0.454545,1.0,1.0,0.8
1464,1465,0.588235,0.75,0.075342,0.017318,1.0,0.0,0.333333,0.0,1.0,0.0,0.916667,0.250,0.285714,1.0,0.285714,0.777778,0.500,0.138686,0.322581,0.2,0.142857,0.428571,0.400000,0.0000,0.666667,1.0,0.4,0.666667,1.0,1.0,0.0,0.046598,1.0,0.000000,0.435360,0.209493,0.2,0.0,1.0,1.0,0.198698,0.000000,0.0,0.178222,0.0,0.0,0.50,0.0,0.250,0.333333,0.666667,0.230769,1.0,0.00,0.5,0.2,0.694268,0.5,0.4,0.340054,1.0,1.0,1.0,0.000000,0.110512,0.0,0.0,0.250000,0.0,0.000000,0.000000,1.0,1.0,0.8


## Feature selection
  * It's a process for selecting best feature and deleting the less important feature to reduce data complexity and dimensionality
  * It will help model to learn effectively

### Variance Threshold
 * This method is filter method
 * It set a threshold value and remove all the features below the threshold value
 * It calculate the variance of a column
 * If data is mostly similar then variance is low, near to zero variance
 * Unsupervised method , no relationship with target variable
 * It should be applied on x_train
 * Applied on Numerical data
    

In [104]:
# importing variance threshold
from sklearn.feature_selection import VarianceThreshold

In [154]:
X_train = processed_train.copy()

In [224]:
# Fit to model
var = VarianceThreshold(threshold = 0.01)
selected = var.fit(X_train)

In [226]:
selected.get_support()

array([ True,  True,  True, False, False, False,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False, False,  True,  True,  True, False,  True, False, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False,  True,
        True,  True,  True])

In [228]:
## seleceted columns
selected_features = X_train.columns[selected.get_support()]

In [218]:
new_x_train = X_train[selected_features]

In [220]:
new_x_train.head()

,Id,MSSubClass,MSZoning,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,HeatingQC,CentralAir,Electrical,2ndFlrSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
0,1,0.235294,0.75,1.0,1.0,1.0,0.0,0.208333,0.250,0.0,0.714286,0.666667,0.500,0.043796,0.112903,0.2,0.857143,0.866667,0.12250,0.666667,1.0,0.4,0.666667,1.000000,1.000000,0.4,1.0,0.0,0.064212,0.0,1.0,1.0,0.413559,0.333333,0.0,0.50,0.5,0.375,0.666667,0.461538,1.0,0.00,0.5,0.2,0.5,0.4,0.368280,1.0,1.0,1.0,0.090909,0.50,1.0,0.8
1,2,0.000000,0.75,1.0,1.0,0.5,0.0,1.000000,0.125,0.0,0.285714,0.555556,0.875,0.233577,0.532258,0.2,0.571429,0.533333,0.00000,1.000000,1.0,0.2,0.666667,1.000000,0.333333,0.0,1.0,0.0,0.121575,0.0,1.0,1.0,0.000000,0.000000,0.5,0.50,0.0,0.375,1.000000,0.307692,1.0,0.25,1.0,0.2,0.5,0.4,0.309140,1.0,1.0,1.0,0.363636,0.25,1.0,0.8
2,3,0.235294,0.75,0.0,1.0,1.0,0.0,0.208333,0.250,0.0,0.714286,0.666667,0.500,0.058394,0.129032,0.2,0.857143,0.866667,0.10125,0.666667,1.0,0.4,0.666667,1.000000,0.666667,0.4,1.0,0.0,0.185788,0.0,1.0,1.0,0.419370,0.333333,0.0,0.50,0.5,0.375,0.666667,0.307692,1.0,0.25,1.0,0.2,0.5,0.4,0.408602,1.0,1.0,1.0,0.727273,0.50,1.0,0.8
3,4,0.294118,0.75,0.0,1.0,0.0,0.0,0.250000,0.250,0.0,0.714286,0.666667,0.500,0.671533,0.612903,0.2,0.928571,1.000000,0.00000,1.000000,1.0,0.0,1.000000,0.333333,1.000000,0.0,1.0,0.0,0.231164,0.5,1.0,1.0,0.366102,0.333333,0.0,0.25,0.0,0.375,0.666667,0.384615,1.0,0.25,0.5,1.0,1.0,0.6,0.431452,1.0,1.0,1.0,0.090909,0.00,1.0,0.0
4,5,0.235294,0.75,0.0,1.0,0.5,0.0,0.625000,0.250,0.0,0.714286,0.777778,0.500,0.065693,0.161290,0.2,0.857143,0.866667,0.21875,0.666667,1.0,0.4,0.666667,1.000000,0.000000,0.4,1.0,0.0,0.209760,0.0,1.0,1.0,0.509927,0.333333,0.0,0.50,0.5,0.500,0.666667,0.538462,1.0,0.25,1.0,0.2,0.5,0.6,0.561828,1.0,1.0,1.0,1.000000,0.50,1.0,0.8


In [222]:
new_x_train.shape

(1460, 54)

In [230]:
X_test = processed_test.copy()

In [232]:
## selecting the same feature for X_test dataset
new_x_test = X_test[selected_features]

In [234]:
new_x_test.shape

(1459, 54)

In [238]:
new_x_test.head()

,Id,MSSubClass,MSZoning,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,HeatingQC,CentralAir,Electrical,2ndFlrSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,MoSold,YrSold,SaleType,SaleCondition
1460,1461,0.000000,0.50,1.0,1.000000,1.0,0.0,0.500000,0.125,0.0,0.285714,0.444444,0.625,0.364964,0.822581,0.2,0.857143,0.866667,0.0000,1.000000,1.0,0.2,1.000000,1.0,1.0,0.8,0.6,0.094364,0.115582,1.0,1.0,1.0,0.000000,0.0,0.0,0.25,0.0,0.250,1.000000,0.230769,1.0,0.00,0.5,0.2,1.0,0.2,0.490591,1.0,1.0,1.0,0.454545,1.0,1.0,0.8
1461,1462,0.000000,0.75,0.0,1.000000,0.0,0.0,0.500000,0.250,0.0,0.285714,0.555556,0.625,0.386861,0.870968,0.6,0.928571,0.933333,0.0675,1.000000,1.0,0.2,1.000000,1.0,1.0,0.0,1.0,0.000000,0.173801,1.0,1.0,1.0,0.000000,0.0,0.0,0.25,0.5,0.375,0.666667,0.307692,1.0,0.00,0.5,0.2,1.0,0.2,0.209677,1.0,1.0,1.0,0.454545,1.0,1.0,0.8
1462,1463,0.235294,0.75,0.0,1.000000,1.0,0.0,0.333333,0.250,0.0,0.714286,0.444444,0.500,0.102190,0.225806,0.2,0.857143,0.866667,0.0000,1.000000,1.0,0.4,0.666667,1.0,1.0,0.4,1.0,0.000000,0.058647,0.5,1.0,1.0,0.339467,0.0,0.0,0.50,0.5,0.375,1.000000,0.307692,1.0,0.25,1.0,0.2,0.0,0.4,0.323925,1.0,1.0,1.0,0.181818,1.0,1.0,0.8
1463,1464,0.235294,0.75,0.0,1.000000,1.0,0.0,0.333333,0.250,0.0,0.714286,0.555556,0.625,0.094891,0.225806,0.2,0.857143,0.866667,0.0125,1.000000,1.0,0.4,1.000000,1.0,1.0,0.4,1.0,0.000000,0.138699,0.0,1.0,1.0,0.328329,0.0,0.0,0.50,0.5,0.375,0.666667,0.384615,1.0,0.25,0.5,0.2,0.0,0.4,0.315860,1.0,1.0,1.0,0.454545,1.0,1.0,0.8
1464,1465,0.588235,0.75,0.0,0.333333,1.0,0.0,0.916667,0.250,1.0,0.285714,0.777778,0.500,0.138686,0.322581,0.2,0.428571,0.400000,0.0000,0.666667,1.0,0.4,0.666667,1.0,1.0,0.0,1.0,0.000000,0.435360,0.0,1.0,1.0,0.000000,0.0,0.0,0.50,0.0,0.250,0.666667,0.230769,1.0,0.00,0.5,0.2,0.5,0.4,0.340054,1.0,1.0,1.0,0.000000,1.0,1.0,0.8
